CRSSNT 结果分析

修改bedpe文件内容

In [18]:
import os
os.chdir("D:/360MoveData/Users/dgwei/Desktop/")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
matrix1 = pd.read_csv('PEDV.mergedgap1.cliques.t_o0.1_dg.bedpe', names=['chr1', 's1', 'e1','chr2','s2','e2','info','count','strand1','strand2'], sep='\t')

In [1]:
matrix2 = matrix1[matrix1['s2'] != matrix1['e2']]
matrix3 = matrix2[matrix2['info'].str.split(',').str[-1].astype(float) >= 0.001]
matrix3['info'] = matrix3['info'].str.split(',').str[0]+'-'+matrix3['info'].str.split(',').str[1]
# matrix3

NameError: name 'matrix1' is not defined

In [21]:
matrix3.to_csv("PEDV.mergedgap1.cliques.t_o0.1_dg.0.01.bedpe",sep='\t',index=False,header=False)

CRSSANT 使用笔记，以 PEDV vRIC-seq 数据为例 —— 2024.3.24

1.数据清洗
fastp -i E100035064_L01_116_1.fq -I E100035064_L01_116_2.fq -o PEDV.read1.clean.fq -O PEDV.read2.clean.fq -z 4 -q 20 -u 30 -f 10
perl /storx/max/workspace/vRICseq/RICpipe-master/step0.remove_PCR_duplicates/scripts/remove_duplicated_reads_PE.pl PEDV.read1.clean.fq PEDV.read2.clean.fq PEDV.read1.clean.rm.fq PEDV.read2.clean.rm.fq

2.数据比对
mkdir align_to_reference_genome  #建索引 STAR 2.7.1a
STAR --runMode genomeGenerate --runThreadN 16 --genomeSAindexNbases 6 --genomeDir align_to_reference_genome --genomeFastaFiles MK584552.1.fasta
STAR --runMode alignReads --genomeDir align_to_reference_genome/ --readFilesIn PEDV.read1.clean.rm.fq PEDV.read2.clean.rm.fq --outFileNamePrefix PEDV --runThreadN 50 --genomeLoad NoSharedMemory --outReadsUnmapped Fastx --outFilterMultimapNmax 10 --outFilterScoreMinOverLread 0 --outFilterMatchNminOverLread 0 --outSAMattributes All --outSAMtype BAM Unsorted SortedByCoordinate --alignIntronMin 1 --scoreGap 0 --scoreGapNoncan 0 --scoreGapGCAG 0 --scoreGapATAC 0 --scoreGenomicLengthLog2scale -1 --chimFilter None --chimOutType WithinBAM HardClip --chimSegmentMin 5 --chimJunctionOverhangMin 5 --chimScoreJunctionNonGTAG 0 -- chimScoreDropMax 80 --chimNonchimScoreDropMin 20 --limitOutSJcollapsed 10000000 --limitIObufferSize 1500000000 --limitBAMsortRAM 1154911447

STAR --runMode alignReads --genomeDir align_to_reference_genome/ --readFilesIn PEDV.read1.clean.rm.fq PEDV.read2.clean.rm.fq --outFileNamePrefix PEDV --runThreadN 100 --genomeLoad NoSharedMemory --outReadsUnmapped Fastx --outFilterMultimapNmax 10 --outFilterScoreMinOverLread 0 --outFilterMatchNminOverLread 0 --outSAMattributes All  --alignIntronMin 1 --scoreGap 0 --scoreGapNoncan 0 --scoreGapGCAG 0 --scoreGapATAC 0 --scoreGenomicLengthLog2scale -1 --chimFilter None --chimSegmentMin 5 --chimJunctionOverhangMin 5 --chimScoreJunctionNonGTAG 0 -- chimScoreDropMax 80 --chimNonchimScoreDropMin 20 --limitOutSJcollapsed 10000000 --limitIObufferSize 1500000000

3.Rearrange softclipped alignments and remap

samtools view -h PEDVAligned.out.bam > PEDVAligned.out.sam
报错：[E::sam_format1] Corrupted aux data for read E100035064L1C042R01202759559
samtools view: writing to standard output failed: Invalid argument
解决方法：
https://github.com/LuyiTian/scPipe/issues/129
samtools view PEDVAligned.out.bam | grep -G 'E100035064L1C042R01202759559'  #查看问题行
① STAR 比对直接生成 sam 文件
STAR --runMode alignReads --genomeDir align_to_reference_genome/ --readFilesIn PEDV.read1.clean.rm.fq PEDV.read2.clean.rm.fq --outFileNamePrefix PEDV --runThreadN 100 --genomeLoad NoSharedMemory --outReadsUnmapped Fastx --outFilterMultimapNmax 10 --outFilterScoreMinOverLread 0 --outFilterMatchNminOverLread 0 --outSAMattributes All  --alignIntronMin 1 --scoreGap 0 --scoreGapNoncan 0 --scoreGapGCAG 0 --scoreGapATAC 0 --scoreGenomicLengthLog2scale -1 --chimFilter None --chimSegmentMin 5 --chimJunctionOverhangMin 5 --chimScoreJunctionNonGTAG 0 -- chimScoreDropMax 80 --chimNonchimScoreDropMin 20 --limitOutSJcollapsed 10000000 --limitIObufferSize 1500000000

②可能是 fastp 的问题，考虑重新使用之前数据清洗方法
trimmomatic PE -phred33 -threads 100 E100035064_L01_116_1.fq E100035064_L01_116_2.fq read1.clean.pair.fq read1.clean.unpair.fq read2.clean.pair.fq read2.clean.unpair.fq ILLUMINACLIP:TruSeq3-PE-2.fa:2:30:7:8:true LEADING:25 TRAILING:20 SLIDINGWINDOW:4:15 MINLEN:30
****************************************************
TrimmomaticPE: Started with arguments:
 -phred33 -threads 100 E100035064_L01_116_1.fq E100035064_L01_116_2.fq read1.clean.pair.fq read1.clean.unpair.fq read2.clean.pair.fq read2.clean.unpair.fq ILLUMINACLIP:TruSeq3-PE-2.fa:2:30:7:8:true LEADING:25 TRAILING:20 SLIDINGWINDOW:4:15 MINLEN:30
Using PrefixPair: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT' and 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
Using Long Clipping Sequence: 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTA'
Using Long Clipping Sequence: 'AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC'
Using Long Clipping Sequence: 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
Using Long Clipping Sequence: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT'
ILLUMINACLIP: Using 1 prefix pairs, 4 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Input Read Pairs: 58059581 Both Surviving: 57963711 (99.83%) Forward Only Surviving: 64607 (0.11%) Reverse Only Surviving: 31025 (0.05%) Dropped: 238 (0.00%)
TrimmomaticPE: Completed successfully
*****************************************************
perl /storx/max/workspace/vRICseq/RICpipe-master/step0.remove_PCR_duplicates/scripts/remove_duplicated_reads_PE.pl read1.clean.pair.fq read2.clean.pair.fq read1.clean.pair.rmDup.fq read2.clean.pair.rmDup.fq
****************************************************
cutadapt -j 10 -b A{100} -b C{100} -b G{100} -b T{100} -n 3 -m 30 -e 0.1 -o read1.clean.rmDup.rmPoly.fq read1.clean.pair.rmDup.fq
cutadapt -j 10 -b A{100} -b C{100} -b G{100} -b T{100} -n 3 -m 30 -e 0.1 -o read2.clean.rmDup.rmPoly.fq read2.clean.pair.rmDup.fq
****************************************************
EXITING because of FATAL ERROR in reads input: quality string length is not equal to sequence length
@E100035064L1C042R04004009235/1
+

SOLUTION: fix your fastq file
这种方法难以解决问题，似乎是因为 trimmomatic 会修改 fastq 文件

python ../scripts/softreverse.py PEDVAligned.out.sam PEDV.softrev.fastq

比对反向 fastq 文件
STAR --runMode alignReads --genomeDir align_to_reference_genome/ --readFilesIn PEDV.softrev.fastq --outFileNamePrefix PEDV.softrev --runThreadN 100 --genomeLoad NoSharedMemory --outReadsUnmapped Fastx --outFilterMultimapNmax 10 --outFilterScoreMinOverLread 0 --outFilterMatchNminOverLread 0 --outSAMattributes All  --alignIntronMin 1 --scoreGap 0 --scoreGapNoncan 0 --scoreGapGCAG 0 --scoreGapATAC 0 --scoreGenomicLengthLog2scale -1 --chimFilter None  --chimSegmentMin 5 --chimJunctionOverhangMin 5 --chimScoreJunctionNonGTAG 0 -- chimScoreDropMax 80 --chimNonchimScoreDropMin 20 --limitOutSJcollapsed 10000000 --limitIObufferSize 1500000000

合并两个 sam 文件
samtools merge PEDV.merged.sam PEDVAligned.out.sam PEDV.softrev.Aligned.out.sam

Classify alignments #对合并得到的 sam 文件进行分类
python ../scripts/gaptypes.py PEDV.merged.sam PEDV.merge -1 15 100
******************************************************
Total input alignment number: 71615090
           Continuous alignments (no gaps): 61450222
             Two-segment gapped alignments: 9755973
           Multi-segment gapped alignments: 353145
        Other chimeric (different str/chr): 0
          Overlapping chimeric (homotypic): 0
                            Bad alignments: 33088
******************************************************

Segment and gap statistics
filetype: 'sam' file or text file containing 'list' of lengths
gaptype: 'min', only the shortest gap in the alignment, or 'all' for all gaps
******************************************************
python ../../scripts/gaplendist.py PEDV.mergedgap1.sam sam PEDV.mergedgap1.min.list min
python ../../scripts/gaplendist.py PEDV.mergedgap1.min.list list PEDV.mergedgap1.min.list.pdf min
python ../../scripts/gaplendist.py PEDV.mergedgapm.sam sam PEDV.mergedgapm.min.list min
python ../../scripts/gaplendist.py PEDV.mergedgapm.min.list list PEDV.mergedgapm.min.list.pdf min

python ../../scripts/seglendist.py PEDV.mergedgap1.sam sam PEDV.mergedgap1.seglendist.list
python ../../scripts/seglendist.py PEDV.mergedgap1.seglendist.list list PEDV.mergedgap1.seglendist.pdf


Filter spliced and short gaps
Output files gap1.sam and gapm.sam may contain alignments that have only splicing junctions and short 1-2 nt gaps due to artifacts. These are filtered out using gapfilter.py before further processing Splicing junctions and short gaps in other output files can be safely ignored. The annotation file containing the splicing junctions should be in GTF format. idloc, location of the transcript_ID field, is usually field 11. short is set to either yes which means 'remove short 1-2nt gaps', or no, which means 'ignore short 1-2nt gaps
******************************************************
python ../../scripts/gapfilter.py AJ1102-MK584552.1.gtf PEDV.mergedgap1.sam PEDV.mergedgap1.filter.sam 15 yes
('2024-03-25 08:41:31', 'Starting the gapfilter analysis ...')
('2024-03-25 08:41:31', 'Reading the annotation GTF file ...')
('2024-03-25 08:41:31', 'Building the splicing junction database ...')
('2024-03-25 08:41:31', 'Started filtering splice junctions ...')
('2024-03-25 08:41:51', 'Processed', 1000000, 'reads ...')
('2024-03-25 08:42:11', 'Processed', 2000000, 'reads ...')
('2024-03-25 08:42:29', 'Processed', 3000000, 'reads ...')
('2024-03-25 08:42:48', 'Processed', 4000000, 'reads ...')
('2024-03-25 08:43:05', 'Processed', 5000000, 'reads ...')
('2024-03-25 08:43:24', 'Processed', 6000000, 'reads ...')
('2024-03-25 08:43:42', 'Processed', 7000000, 'reads ...')
('2024-03-25 08:44:01', 'Processed', 8000000, 'reads ...')
('2024-03-25 08:44:27', 'Processed', 9000000, 'reads ...')
('\n                        Total alignments:', 9755973)
('                   All gapped alignments:', 9755973)
('    Alignments with at least 1 good gaps:', 7925579)
('    Alignments with at least 2 good gaps:', 83249)
('  Number of annotated splicing junctions:', 1, '\n')
('2024-03-25 08:44:46', 'Finished gapfilter.py successfully\n')
python ../../scripts/gapfilter.py AJ1102-MK584552.1.gtf PEDV.mergedgapm.sam PEDV.mergedgapm.filter.sam 15 yes
('2024-03-25 08:47:46', 'Starting the gapfilter analysis ...')
('2024-03-25 08:47:46', 'Reading the annotation GTF file ...')
('2024-03-25 08:47:46', 'Building the splicing junction database ...')
('2024-03-25 08:47:46', 'Started filtering splice junctions ...')
('\n                        Total alignments:', 353145)
('                   All gapped alignments:', 353145)
('    Alignments with at least 1 good gaps:', 338644)
('    Alignments with at least 2 good gaps:', 244064)
('  Number of annotated splicing junctions:', 1, '\n')
('2024-03-25 08:47:57', 'Finished gapfilter.py successfully\n')

Cluster gap1 and trans alignments to DGs
samtools view -b -S PEDV.mergedgap1.sam  | samtools sort -o PEDV.mergedgap1.sorted.bam
bedtools genomecov -bg -split -strand + -ibam PEDV.mergedgap1.sorted.bam -g PEDV.chrom.sizes > PEDV.mergedgap1_plus.bedgraph
bedtools genomecov -bg -split -strand - -ibam PEDV.mergedgap1.sorted.bam -g PEDV.chrom.sizes > PEDV.mergedgap1_minus.bedgraph
# gtf 格式转 bed 格式
convert2bed -i gtf -o bed <AJ1102-MK584552.1.gtf > AJ1102-MK584552.1.bed

python ../../scripts/crssant.py -out DGs/ -cluster cliques PEDV.mergedgap1.sam AJ1102-MK584552.1.bed PEDV.mergedgap1_plus.bedgraph,PEDV.mergedgap1_minus.bedgraph -n 100
**************************************************************************
Traceback (most recent call last):
  File "/store/max/anaconda3/envs/CRSSANT/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "../../scripts/crssant.py", line 701, in run_analysis
    dg_stats_dict = create_stats(dg_filtered_dict,genealign,covdict) #not yet
  File "../../scripts/crssant.py", line 550, in create_stats
    overlap2=max([covdict[(chrom2,strand2,i)] for i in range2 if
ValueError: max() arg is an empty sequence
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "../../scripts/crssant.py", line 752, in <module>
    main()
  File "../../scripts/crssant.py", line 745, in main
    for (genepair,dg_filtered_dict,dg_stats_dict) in results:
  File "/store/max/anaconda3/envs/CRSSANT/lib/python3.6/multiprocessing/pool.py", line 735, in next
    raise value
ValueError: max() arg is an empty sequence
**************************************************************************
python ../../../scripts/gapmcluster.py PEDV.mergedgap1.cliques.t_o0.1_dg.bedpe ../PEDV.mergedgapm.filter.sam #得到动态结构

python ../../../scripts/bedpetobed12.py PEDV.mergedgap1.cliques.t_o0.1_dg.bedpe PEDV.mergedgap1.cliques.t_o0.1_dg.bed
sortBed -i ACTB.cliques.t_o0.1_dg.bed > ACTB.cliques.t_o0.1_dg_sorted.bed
输出文件的首行添加 track graphType=arc

# sam to sorted bam
samtools view -b -S PEDV.mergedgapm.filter_tg.sam | samtools sort -o PEDV.mergedgapm.filter_tg.sorted.bam
samtools index PEDV.mergedgapm.filter_tg.sorted.bam

In [15]:
#从 PEDV.mergedgapm.filter_tg.sam 文件中提取出信息
import os
os.chdir("D:/360MoveData/Users/dgwei/Desktop/")
import math

In [17]:
with open ('PEDV.mergedgapm.filter_tg.txt','w') as f :
    with open ('PEDV.mergedgapm.filter_tg.sam','r') as F :
        for line in F:
            if line.startswith("@"):
                pass
            else :
                aimline = line.split('\t')[-1].rstrip()
                DGcov1 = aimline.split('_')[1]
                DGcov2 = aimline.split('_')[3]
                TGcov = int(math.sqrt(int(DGcov1)*int(DGcov2)))
                f.write(aimline+'\t'+str(TGcov)+'\n')



In [74]:
#处理 PEDV.mergedgapm.filter_tg.txt 文件
import pandas as pd
import os
os.chdir("D:/360MoveData/Users/dgwei/Desktop/")

In [77]:
genome_dict={}
with open('MK584552.1.fasta', 'r') as F:
    for line in F:
        if line.startswith(">"):
            key = line.split(" ")[0].strip(">\n")
            genome_dict[key] = ""
        else:
            genome_dict[key] += line.rstrip().upper()

In [88]:
for key,value in genome_dict.items():
    line1 = value[25438:25463]
    line2 = value[26268:26355]
    line3 = line1 + line2
line3


'GACGAGTATGCTACAATTAGTGAATGCTTTCTATGTCCGGTCAAAACACGGCGACTACTCAGCTGTGAGTAATCCGAGTGCGGTTCTCACAGATAGTGAGAAAGTGCTTCAC'

In [89]:
for key,value in genome_dict.items():
    line4 = value[25275:25297]
    line5 = value[26297:26407]
    line6 = line1 + line2
line6

'GACGAGTATGCTACAATTAGTGAATGCTTTCTATGTCCGGTCAAAACACGGCGACTACTCAGCTGTGAGTAATCCGAGTGCGGTTCTCACAGATAGTGAGAAAGTGCTTCAC'

In [91]:
line7 = line3 + line6
line7

'GACGAGTATGCTACAATTAGTGAATGCTTTCTATGTCCGGTCAAAACACGGCGACTACTCAGCTGTGAGTAATCCGAGTGCGGTTCTCACAGATAGTGAGAAAGTGCTTCACGACGAGTATGCTACAATTAGTGAATGCTTTCTATGTCCGGTCAAAACACGGCGACTACTCAGCTGTGAGTAATCCGAGTGCGGTTCTCACAGATAGTGAGAAAGTGCTTCAC'